# Getting started with Julia

Because JuMP is embedded in Julia, knowing some basic Julia is important
before you start learning JuMP.
The content is based on https://jump.dev/JuMP.jl/stable/tutorials/getting_started/getting_started_with_julia/ (*generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*).

## The package manager

### Installing packages

No matter how wonderful Julia's base language is, at some point you will want
to use an extension package.  Some of these are built-in, for example random
number generation is available in the `Random` package in the standard
library. These packages are loaded with the commands `using` and `import`.

In [ ]:
using Random  # The equivalent of Python's `from Random import *`
import Random  # The equivalent of Python's `import Random`

Random.seed!(33)

[rand() for i in 1:10]

The Package Manager is used to install packages that are not part of Julia's
standard library.

For example the following can be used to install JuMP,
```julia
using Pkg
Pkg.add("JuMP")
```

For a complete list of registered Julia packages see the package listing at
[JuliaHub](https://juliahub.com).

From time to you may wish to use a Julia package that is not registered.  In
this case a git repository URL can be used to install the package.
```julia
using Pkg
Pkg.add("https://github.com/user-name/MyPackage.jl.git")
```

### Package environments

By default, `Pkg.add` will add packages to Julia's global environment.
However, Julia also has built-in support for virtual environments.
You can see what packages are installed in the current environment with `Pkg.status()`.

In [ ]:
using Pkg
Pkg.status()

Activate a virtual environment with:
```julia
import Pkg; Pkg.activate("/path/to/environment")
```

The path `"."` corresponds to the output of `pwd()` (that you can change with `cd()`)

In [ ]:
@show pwd()
cd("..")
@show pwd()

The path `@__DIR__` corresponds to the folder containing this notebook:

In [ ]:
@__DIR__

We will use `@__DIR__` as the `Project.toml` file is in this folder.

In [ ]:
Pkg.activate(@__DIR__)

We can see now that the environment has changed.

In [ ]:
Pkg.status()

Running `Pkg.instantiate()` will:
* install the packages of `Project.toml` along with their dependencies and
* precompile the commonly used methods of the packages.

In [ ]:
Pkg.instantiate()

This creates a `Manifest.toml` file containing the versions of all packages used.

When a `Manifest.toml` exists:
* `Pkg.instantiate()` follows the versions in that file and
* `Pkg.update()` tries to find more recent versions.

We _strongly_ recommend you create a Pkg environment for each project
that you create in Julia, and add only the packages that you need, instead
of adding lots of packages to the global environment. The [Pkg manager documentation](https://julialang.github.io/Pkg.jl/v1/environments/)
has more information on this topic.

## Numbers and arithmetic

Since we want to solve optimization problems, we're going to be using a lot of
math. Luckily, Julia is great for math, with all the usual operators:

In [ ]:
@show 1 + 1
@show 1 - 2
@show 2 * 2
@show 4 / 5
@show 3^2

The `@` in front of something indicates that it is a macro, which is
a special type of function. In this case, `@show` prints the expression as
typed (e.g., `1 - 2`), as well as the evaluation of the expression (`-1`).

Did you notice how Julia didn't print `.0` after some of the numbers? Julia is
a dynamic language, which means you never have to explicitly declare the type
of a variable. However, in the background, Julia is giving each variable a
type. Check the type of something using the `typeof` function:

In [ ]:
@show typeof(1)
@show typeof(1.0)

Here `1` is an `Int64`, which is an integer with 64 bits of precision, and
`1.0` is a `Float64`, which is a floating point number with 64-bits of
precision.

We create complex numbers using `im`:

In [ ]:
x = 2 + 1im
@show real(x)
@show imag(x)
@show typeof(x)
@show x * (1 - 2im)

The curly brackets surround what we call the _parameters_ of a type. You
can read `Complex{Int64}`  as "a complex number, where the real and
imaginary parts are represented by `Int64`." If we call
`typeof(1.0 + 2.0im)` it will be `Complex{Float64}`, which a complex
number with the parts represented by `Float64`.

There are also some cool things like an irrational representation of π.

In [ ]:
π

To make π (and most other Greek letters), type `\pi` and then press `[TAB]`.

In [ ]:
typeof(π)

However, if we do math with irrational numbers, they get converted to
`Float64`:

In [ ]:
a = 2π / 3
@show typeof(a)
a

When using numbers supporting more accuray, `π` is more accurate. Hence the avantage of the `Irrational` type.

In [ ]:
a = big(2) * π / big(3)
@show typeof(a)
a

### Floating point numbers

A `Float64` is a [floating point](https://en.wikipedia.org/wiki/Floating-point_arithmetic)
approximation of a real number using 64-bits of information.

Because it is an approximation, things we know hold true in mathematics don't
hold true in a computer! For example:

In [ ]:
0.1 * 3 == 0.3

In [ ]:
sin(2π / 3) == √3 / 2

Get `√` by typing `\sqrt` then press `[TAB]`.

Let's see what the differences are:

In [ ]:
0.1 * 3 - 0.3

In [ ]:
sin(2π / 3) - √3 / 2

They are small, but not zero!

One way of explaining this difference is to consider how we would write
`1 / 3` and `2 / 3` using only four digits after the decimal point. We would
write `1 / 3` as `0.3333`, and `2 / 3` as `0.6667`. So, despite the fact that
`2 * (1 / 3) == 2 / 3`, `2 * 0.3333 == 0.6666 != 0.6667`.

Let's try that again using ≈ (`\approx + [TAB]`) instead of `==`:

In [ ]:
0.1 * 3 ≈ 0.3

In [ ]:
sin(2π / 3) ≈ √3 / 2

`≈` is a clever way of calling the `isapprox` function:

In [ ]:
isapprox(sin(2π / 3), √3 / 2; atol = 1e-8)

Floating point is the reason solvers use tolerances when they solve
optimization models. A common mistake you're likely to make is checking
whether a binary variable is 0 using `value(z) == 0`. Always remember to
use something like `isapprox` when comparing floating point numbers.

Note that `isapprox` will always return `false` if one of the number being
compared is `0` and `atol` is zero (its default value).

In [ ]:
1e-300 ≈ 0.0

We can use the `@edit` macro to see what's happening.

In [ ]:
@edit 1e-300 ≈ 0.0

so always set a nonzero value of `atol` if one of the arguments can be zero.

In [ ]:
isapprox(1e-9, 0.0; atol = 1e-8)

Gurobi has a [good series of articles](https://www.gurobi.com/documentation/9.0/refman/num_grb_guidelines_for_num.html)
on the implications of floating point in optimization if you want to read
more.

If you aren't careful, floating point arithmetic can throw up all manner of
issues. For example:

In [ ]:
1 + 1e-16 == 1

It even turns out that floating point numbers aren't associative!

In [ ]:
(1 + 1e-16) - 1e-16 == 1 + (1e-16 - 1e-16)

It's important to note that this issue isn't Julia-specific. It happens in
every programming language (try it out in Python).

## Vectors, matrices and arrays

Similar to Matlab, Julia has native support for vectors, matrices and tensors;
all of which are represented by arrays of different dimensions. Vectors are
constructed by comma-separated elements surrounded by square brackets:

In [ ]:
b = [5, 6]

Matrices can be constructed with spaces separating the columns, and semicolons
separating the rows:

In [ ]:
A = [1.0 2.0; 3.0 4.0]

We can do linear algebra:

In [ ]:
x = A \ b

Here is floating point at work again! `x` is approximately `[-4, 4.5]`.

In [ ]:
A * x

In [ ]:
A * x ≈ b

Note that when multiplying vectors and matrices, dimensions matter. For
example, you can't multiply a vector by a vector:

In [ ]:
b * b

But multiplying transposes works:

In [ ]:
b' * b

In [ ]:
b * b'

## Other common types

### Strings

Double quotes are used for strings:

In [ ]:
typeof("This is Julia")

Unicode is fine in strings:

In [ ]:
typeof("π is about 3.1415")

Use [`println`](https://docs.julialang.org/en/v1/base/io-network/#Base.println)
to print a string:

In [ ]:
println("Hello, World!")

Use `$()` to interpolate values into a string:

In [ ]:
x = 123
println("The value of x is: $(x)")

### Symbols

Julia `Symbol`s are a data structure from the compiler that represent Julia
identifiers (i.e., variable names).

In [ ]:
println("The value of x is: $(eval(:x))")

We used `eval` here to demonstrate how Julia links `Symbol`s to variables.
However, avoid calling `eval` in your code. It is usually a sign that your
code is doing something that could be more easily achieved a different
way. The [Community Forum](https://discourse.julialang.org/c/domain/opt/13)
is a good place to ask for advice on alternative approaches.

In [ ]:
typeof(:x)

You can think of a `Symbol` as a `String` that takes up less memory, and that
can't be modified.

Convert between `String` and `Symbol` using their constructors:

In [ ]:
String(:abc)

In [ ]:
Symbol("abc")

`Symbol`s are often (ab)used to stand in for a `String` or an `Enum`, when
one of the former is likely a better choice. The JuMP Style guide
recommends reserving `Symbol`s for identifiers.

### Tuples

Julia makes extensive use of a simple data structure called Tuples. Tuples are
immutable collections of values. For example:

In [ ]:
t = ("hello", 1.2, :foo)

In [ ]:
typeof(t)

Tuples can be accessed by index, similar to arrays:

In [ ]:
t[2]

And they can be "unpacked" like so:

In [ ]:
a, b, c = t
b

The values can also be given names, which is a convenient way of making
light-weight data structures.

In [ ]:
t = (word = "hello", num = 1.2, sym = :foo)

Values can be accessed using dot syntax:

In [ ]:
t.word

## Dictionaries

Similar to Python, Julia has native support for dictionaries. Dictionaries
provide a very generic way of mapping keys to values.  For example, a map of
integers to strings:

In [ ]:
d1 = Dict(1 => "A", 2 => "B", 4 => "D")

Type-stuff again: `Dict{Int64,String}` is a dictionary with `Int64` keys and `String` values.

Looking up a values uses the bracket syntax:

In [ ]:
d1[2]

Dictionaries support non-integer keys and can mix data types:

In [ ]:
Dict("A" => 1, "B" => 2.5, "D" => 2 - 3im)

Julia types form a hierarchy. Here the value type of the dictionary is
`Number`, which is a generalization of `Int64`, `Float64`, and
`Complex{Int}`. Leaf nodes in this hierarchy are called "concrete" types,
and all others are called "Abstract". In general, having variables with
abstract types like `Number` can lead to slower code, so you should try to
make sure every element in a dictionary or vector is the same type. For
example, in this case we could represent every element as a
`Complex{Float64}`:

In [ ]:
Dict("A" => 1.0 + 0.0im, "B" => 2.5 + 0.0im, "D" => 2.0 - 3.0im)

Dictionaries can be nested:

In [ ]:
d2 = Dict("A" => 1, "B" => 2, "D" => Dict(:foo => 3, :bar => 4))

In [ ]:
d2["B"]

In [ ]:
d2["D"][:foo]

## Structs

You can define custom datastructures with `struct`:

In [ ]:
struct MyStruct
    x::Int
    y::String
    z::Dict{Int,Int}
end

a = MyStruct(1, "a", Dict(2 => 3))

In [ ]:
a.x

By default, these are not mutable

In [ ]:
a.x = 2

However, you can declare a `mutable struct` which is mutable:

In [ ]:
mutable struct MyStructMutable
    x::Int
    y::String
    z::Dict{Int,Int}
end

a = MyStructMutable(1, "a", Dict(2 => 3))
a.x

In [ ]:
a.x = 2

a

## Loops

Julia has native support for for-each style loops with the syntax
`for <value> in <collection> end`:

In [ ]:
for i in 1:5
    println(i)
end

!!! info
    Ranges are constructed as `start:stop`, or `start:step:stop`.

In [ ]:
for i in 1.2:1.1:5.6
    println(i)
end

This for-each loop also works with dictionaries:

In [ ]:
for (key, value) in Dict("A" => 1, "B" => 2.5, "D" => 2 - 3im)
    println("$(key): $(value)")
end

Note that in contrast to vector languages like Matlab and R, loops do not
result in a significant performance degradation in Julia.

## Control flow

Julia control flow is similar to Matlab, using the keywords
`if-elseif-else-end`, and the logical operators `||` and `&&` for **or** and
**and** respectively:

In [ ]:
for i in 0:5:15
    if i < 5
        println("$(i) is less than 5")
    elseif i < 10
        println("$(i) is less than 10")
    else
        if i == 10
            println("the value is 10")
        else
            println("$(i) is bigger than 10")
        end
    end
end

## Comprehensions

Similar to languages like Haskell and Python, Julia supports the use of simple
loops in the construction of arrays and dictionaries, called comprehensions.

A list of increasing integers:

In [ ]:
[i for i in 1:5]

Matrices can be built by including multiple indices:

In [ ]:
[i * j for i in 1:5, j in 5:10]

Conditional statements can be used to filter out some values:

In [ ]:
[i for i in 1:10 if i % 2 == 1]

A similar syntax can be used for building dictionaries:

In [ ]:
Dict("$(i)" => i for i in 1:10 if i % 2 == 1)

## Functions

A simple function is defined as follows:

In [ ]:
function print_hello()
    return println("hello")
end
print_hello()

Arguments can be added to a function:

In [ ]:
function print_it(x)
    return println(x)
end
print_it("hello")
print_it(1.234)
print_it(:my_id)

Optional keyword arguments are also possible:

In [ ]:
function print_it(x; prefix = "value:")
    return println("$(prefix) $(x)")
end
print_it(1.234)
print_it(1.234; prefix = "val:")

The keyword `return` is used to specify the return values of a function:

In [ ]:
function mult(x; y = 2.0)
    return x * y
end

mult(4.0)

In [ ]:
mult(4.0; y = 5.0)

### Anonymous functions

The syntax `input -> output` creates an anonymous function. These are most
useful when passed to other functions. For example:

In [ ]:
f = x -> x^2
f(2)

In [ ]:
map(x -> x^2, 1:4)

### Type parameters

We can constrain the inputs to a function using type parameters, which are
`::` followed by the type of the input we want. For example:

In [ ]:
function foo(x::Int)
    return x^2
end

function foo(x::Float64)
    return exp(x)
end

function foo(x::Number)
    return x + 1
end

@show foo(2)
@show foo(2.0)
@show foo(1 + 1im)

But what happens if we call `foo` with something we haven't defined it for?

In [ ]:
foo([1, 2, 3])

We get a dreaded `MethodError`! A `MethodError` means that you passed a
function something that didn't match the type that it was expecting. In this
case, the error message says that it doesn't know how to handle an
`Vector{Int64}`, but it does know how to handle `Float64`, `Int64`, and
`Number`.

Read the "Closest candidates" part of the error message carefully to get a hint as to what was expected.

### Broadcasting

In the example above, we didn't define what to do if `f` was passed a
`Vector`. Luckily, Julia provides a convenient syntax for mapping `f`
element-wise over arrays! Just add a `.` between the name of the function and
the opening `(`. This works for _any_ function, including functions with
multiple arguments. For example:

In [ ]:
f.([1, 2, 3])

Get a `MethodError` when calling a function that takes a `Vector`,
`Matrix`, or `Array`? Try broadcasting it!

## Mutable vs immutable objects

Some types in Julia are *mutable*, which means you can change the values
inside them. A good example is an array. You can modify the contents of an
array without having to make a new array.

In contrast, types like `Float64` are *immutable*. You can't modify the
contents of a `Float64`.

This is something to be aware of when passing types into functions. For
example:

In [ ]:
function mutability_example(mutable_type::Vector{Int}, immutable_type::Int)
    mutable_type[1] += 1
    immutable_type += 1
    return
end

mutable_type = [1, 2, 3]
immutable_type = 1

mutability_example(mutable_type, immutable_type)

println("mutable_type: $(mutable_type)")
println("immutable_type: $(immutable_type)")

Because `Vector{Int}` is a mutable type, modifying the variable inside the
function changed the value outside of the function. In contrast, the change
to `immutable_type` didn't modify the value outside the function.

You can check mutability with the `isimmutable` function:

In [ ]:
isimmutable([1, 2, 3])

In [ ]:
isimmutable(1)